# Discovery of potential microhaplotype markers in _Plasmodium vivax_

The aim of this notebook it to scan the accessible genome and calculate a number of summary statistics (here: cardinality, heterozigosity, entropy) in partially-overlapping sliding windows, to identify potential marker candidates.

This notebook serves two purposes. First, it provides details on the underlying methods used in the main publication: [Lineage-informative microhaplotypes for spatio-temporal surveillance of Plasmodium vivax malaria parasites](https://www.medrxiv.org/content/10.1101/2023.03.13.23287179v1). Second, it provides a template for further microhaplotype candidates selection analysis, for example by changing the selection criteria and/or the statistics calculated for each window.

The notebook consists of four parts:
1. [Setup the environment](#Setup-the-environment)
1. [Load the data](#Load-the-data)
1. [Select the variants](#Select-the-variants)
1. [Calculate summary statistics in sliding windows across the core genome](#Calculate-summary-statistics-in-sliding-windows-across-the-core-genome)

We used data from a subset of high-quality samples part of the open [MalariaGEN Pv4 dataset](https://www.malariagen.net/resource/30), which contains genome variation data on nearly two-thousands worldwide samples of natural _Plasmodium vivax_ infections. Details on this project, the methods used, and all contributing partners can be found in the key publication: [**MalariaGEN et al, Wellcome Open Research 2022, 7:136 https://doi.org/10.12688/wellcomeopenres.17795.1**](https://doi.org/10.12688/wellcomeopenres.17795.1).

The dataset can be accessed in [a number of ways](https://new_malariagen.staging.sanger.ac.uk/resource/30/). Here we used the the [`malariagen_data` Python package](https://github.com/malariagen/malariagen-data-python), which allows to use the data directly from the cloud and without having to first download them locally. The [Pv4 user guide](https://malariagen.github.io/parasite-data/pv4/Data_access.html) provides all the information on how to use the package as well as some examples to get started.

All required files for running this notebook can be found in the **supplementary_files** directory. 

Pre-computed dataframes and output files can be found in the **precomputed** directory.

## Setup the environment

This notebook can be run from any computer and can also work from a compute node within Google Cloud, for example via MyBinder or Google Colab, which are free interactive computing services running in a cloud environment. We primarily use common packages that come pre-installed in many computing environments. However, some specialised packages (e.g. [`malariagen_data`](https://github.com/malariagen/malariagen-data-python)) might require manual installation.

For example, the line below can be uncommented and run on Google Colab to install `malariagen-data` if needed:

In [1]:
#!pip install -q --no-warn-conflicts malariagen-data==7.14.1

**Note:** To mantain compatibility with some on-demand cloud environments, we use specific versions of each package which might not be compatible with the most recent one in your local Python installation. If you having trouble running the code, check to make sure that the versioning of your packages match the ones reported below.

In [12]:
# Load all required packages and print their version to check compatability
from importlib.metadata import version
import os
import glob

print(f'Numpy version: {version("numpy")}')
print(f'Pandas version: {version("pandas")}')
print(f'scikit-allel version: {version("scikit-allel")}')
print(f'malariagen-data version: {version("malariagen_data")}')

import numpy as np
import pandas as pd
import allel
from malariagen_data.pv4 import Pv4

# Suppress warning
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

Numpy version: 1.23.5
Pandas version: 2.2.0
scikit-allel version: 1.3.7
malariagen-data version: 8.6.0


ModuleNotFoundError: No module named 'ipyleaflet'

## Load the data

As previously stated, this notebook accesses the genetic data from the [MalariaGEN Pv4 data resource](https://www.malariagen.net/resource/30) via the [`malariagen_data` Python package](https://github.com/malariagen/malariagen-data-python), which allows you to use the data directly from the cloud and without having to first download them locally. Detailed instructions can be found in th [Pv4 user guide](https://malariagen.github.io/parasite-data/pv4/Data_access.html), which this section is based on.

In [10]:
# Initialise the environment
pv4 = Pv4()

NameError: name 'Pv4' is not defined

First, we load the **sample metadata**, containing details on each sample in the resource such as time and place of collection, quality metrics, etc.

In [ ]:
sample_metadata = pv4.sample_metadata()
sample_metadata.head()

The `sample_metadata` now need to be appended with supplemental information required for sample selection, such as a measure of within-host diversity ($F_{WS}$) available from the [Pv4 data resource](https://new_malariagen.staging.sanger.ac.uk/resource/30/). The ($F_{WS}$) statistic is needed to select samples for downstream analysis that are deemed likely to be clonal lineages (and to exclude likely polyclonal lineages) which simplifies marker selection. 

In [5]:
pv4_fws = pd.read_csv('../supplementary_files/Pv4_fws.txt', sep='\t', comment='t')
sample_metadata = pd.merge(sample_metadata, pv4_fws, on='Sample', how='outer')
pv1_samples = pd.read_csv('../supplementary_files/Samples_in_Pv1.tsv', sep='\t')
sample_metadata = sample_metadata.merge(pv1_samples, on='Sample', how='left')

Second, we load the **variants data**, which includes the details on each individual genetic variation discovered in the complete Pv4 dataset, such as their position in the genome, the type of variation, quality metrics, etc. 

In [6]:
variant_dataset = pv4.variant_calls(extended=True)
variant_dataset

<xarray.Dataset>
Dimensions:                                   (variants: 4571056, alleles: 7,
                                               samples: 1895, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 64, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(65536, 6), meta=np.ndarray>

As you can see above, the `variant_dataset` contains 4,571,056 variants from 1,895 samples. 


Finally, the [Pv4 data resource](https://new_malariagen.staging.sanger.ac.uk/resource/30/) also provides the coordinate boundaries of the _P. vivax_ "core" (or accessible) genome, which are genome regions that consistently have high-quality mapping. These "core" genome regions will be used to limit the discovery of potential marker windows to accessible parts of the genome only, therefore avoiding sub-telomeric and internal hypervariable regions that would be likely too unstable for inclusion in a subsequent marker panel. We can now load the **pv regions** coordinates and then modify the dataframe to align with the `variant_dataset`.

In [7]:
pv_regions = pd.read_csv( "../supplementary_files/Pv4_regions.bed",
                         sep="\t", comment="t", names=["chrom", "chromStart", "chromEnd", "name"])
# The file is 0-based, and needs to be converted to be 1-based for consistency with the variant data
pv_regions[["chromStart", "chromEnd"]] += 1

pv_regions.head()

,chrom,chromStart,chromEnd,name
0,PvP01_01_v1,1,116542,Sub
1,PvP01_01_v1,116542,677963,Core
2,PvP01_01_v1,677963,679790,Cen
3,PvP01_01_v1,679790,903592,Core
4,PvP01_01_v1,903592,1021665,Sub


## Select the variants

Here we decide which variants to include in the microhaplotypes discovery analysis of each sliding window.

### Samples selection

Before selecting variants, we need to decide which samples to include for the downstream analysis, as we will need to calculate the global microhaplotype allele frequencies later in this notebook, among other statistics. These calculations will be made easier by choosing a subset of samples from the [Pv4 data resource] that are unique, high-quality, and monoclonal. In order to select only these samples, we used the following criteria:
* Are **clonal** ($F_{WS}$ > 0.95).
* Have **low missingness** (50% or more of the positions in the genome of the sample has a genotype call (`% callable < 50%`). 
* Are **neither technical nor biological duplicates** (`Exclusion reason == Analysis_set`). This `Analysis_set` flag in the metadata allows you to select only the highest quality sample for downstream analysis in the event of duplicate sample(s) in the [Pv4 data resource](https://new_malariagen.staging.sanger.ac.uk/resource/30/). 
* Due to data restrictions at the time of analysis, these samples are from a limited group of studies.

If you would like to be more/less stringent on your selection criteria, all of these filters can be changed/removed. 

In [8]:
# List of MalariaGEN partner studies with an explicit agreement to participate in this analysis
useable_studies = ['1128-PV-MULTI-GSK', '1154-PV-TH-PRICE', '1157-PV-MULTI-PRICE', 'X0001-PV-MULTI-HUPALO2016','X0002-PV-KH-PAROBEK2016']

# Filter the samples based on the criteria above
loc_filtered_samples = (
    (sample_metadata["Study"].isin(useable_studies) | sample_metadata.in_pv_10)
    & (sample_metadata["Fws"] > 0.95)
    & (sample_metadata["% callable"] > 50)
    & (sample_metadata["Exclusion reason"] == "Analysis_set")
)
subset_metadata = sample_metadata[loc_filtered_samples]
print(len(subset_metadata))

615


As you can see, after applying these filters we are left with **615 samples** that will be used in the downstream marker discovery analysis.

We can now filter the `variant_dataset` to only include these 615 samples to speed-up processing.

In [9]:
variant_dataset_filtered_samples = variant_dataset.isel(samples=loc_filtered_samples)
variant_dataset_filtered_samples

<xarray.Dataset>
Dimensions:                                   (variants: 4571056, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(65536, 6), meta=np.ndarray>

Which gives us `variant_dataset_filtered_samples`.

### Variants selection

Now that we have selected our samples, we can now proceed to variant selection by filtering for only the variants that are:
- High quality variants that **pass** all quality filters (`variant_filter_pass`). 
- Single nucleotide polymorphisms (**SNP**) variants (`variant_is_snp`).
- Are **biallelic** SNPs (`variant_numalt == 1`).
- Found only in **coding regions** (`variant_CDS`).

All filters can be modified/removed, and in this instance we chose to use only high-confidence bilallelic SNPs in coding regions to make microhaplotype data analysis as simple as possible. In theory you could include things like tri-allelic SNPs, insertions/deletions, etc, but as data-handling is already challenging for multi-allelic markers, we chose simplicity where possible. Noncoding regions can be less stable genome regions, so they were also excluded to future-proof panels for longevity (i.e. reducing the likelihood of variants popping up in assay primer regions over time). 

In [10]:
%%time
filters = (
    ((variant_dataset_filtered_samples["variant_numalt"] == 1).data)
    & (variant_dataset_filtered_samples["variant_filter_pass"].data)
    & (variant_dataset_filtered_samples["variant_is_snp"].data)
    & (variant_dataset_filtered_samples["variant_CDS"].data)
)
# This part might take a while but not too long (~minutes)
variant_dataset_filtered = variant_dataset_filtered_samples.isel(variants=filters.compute())
variant_dataset_filtered

CPU times: user 2.71 s, sys: 343 ms, total: 3.06 s
Wall time: 57.8 s


<xarray.Dataset>
Dimensions:                                   (variants: 440222, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(9339, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(9339,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(9339, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(9339, 6), meta=np.ndarray>

This step has now created `variant_dataset_filtered` which now consists of **440,222 variants**. 


Next, we will further select variants that have **global characteristics** that can increase the informativeness of our potential microhaplotype markers. In this case, we would like to enrich for variants that have *high global minor allele frequencies* (> 0.1) to increase the chances that a variant will not be an incredibly rare (and likely uninformative) variant globally. We will also select for variants that have *low global missingness* (< 0.1), which increases confidence that the variant position is well-represented across diverse populations.

To calculate allele frequencies, we need to process the entire _genotype matrix_ (i.e. every genotype call in every sample) which can take a while. Here we use pre-calculated values to speed-up the process, and the code used to generate the files is provided below in case you wish to run it yourself.

**Important note:** Allele frequency and missingness will need to be re-computed if you use different selection criteria for the variants and/or samples.

In [11]:
#%%time
#
### Uncomment in case you want to re-calculate allele frequency and missingness, e.g. for a different samples/variants selection.
### Note that This process can take a while (~10 minutes)
#
### Load the genotype data
#gt = allel.GenotypeDaskArray(variant_dataset_filtered["call_genotype"].data)
#
### Calculate allele frequency
#ac_pop = gt.count_alleles()
#ac_pop_freq = ac_pop.to_frequencies().compute()
#
### Calculate missingness
#freq_missing = gt.count_missing(axis=1).compute() / gt.shape[1]
#
### Save the results to speed-up future re-use
#np.save('../supplementary_files/ac_pop_freq.npy', ac_pop_freq)
#np.save('../supplementary_files/freq_missing.npy', freq_missing)

In [12]:
# Load pre-calculated frequencies
ac_pop_freq = np.load('../supplementary_files/ac_pop_freq.npy')
freq_missing = np.load('../supplementary_files/freq_missing.npy')

We can now finalise the variants selection process and filter based on global frequency and missingness.

In [13]:
pop_freq_filter = (ac_pop_freq.min(axis=1) > 0.1) & (freq_missing < 0.1)
variant_dataset_filtered = variant_dataset_filtered.isel(variants=pop_freq_filter)
variant_dataset_filtered

<xarray.Dataset>
Dimensions:                                   (variants: 13084, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(247, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(247,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(247,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(247,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(247, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(247, 6), meta=np.ndarray>

We can now see that we have significantly decreased the number of variants in `variant_dataset_filtered` based on the above criteria to **13,084 potential variants**. 

## Calculate summary statistics in sliding windows across the core genome

This is the final and principal section of the notebook. After having prepared the data and selected the individual variants to use in the discovery analysis, we slide a 200 base pair window along the core genome and calculate a range of summary statistics on a per-window basis to scan for candidate microhaplotype regions that contain multiple variants.

In this example and following the [main publication](https://www.medrxiv.org/content/10.1101/2023.03.13.23287179v1), we focus on _cardinality_ and _diversity_. Specifically, for each window the code below calculates:
- Number of filtered variants it contains.
- Number and proportion of samples carrying each unique haplotype.
- Entropy, defined as $\sum$ (gt_freqs * $\log(gt\_freqs)$)
- Heterozygosity, defined as 1 - $\sum$ gt_freqs<sup>2

where `gt_freqs` are the frequencies of each unique haplotype.

The functions below can be modified and expanded to calculate different or additional statistics.

Note: to minimise space requirements, we only store windows containing more than 1 variant and that are highly diverse (global heterozygosity $\ge$ 0.5). We chose heterozygosity $\ge$ 0.5 because that is the theoretical maximum heterozygosity possible for a single biallelic SNP, so we were seeking to capture additional informativeness compared to the best-case scenario for a traditional biallelic SNP panel. 

### Utility functions

In [14]:
def filter_variants(variant_dataset, field, value):
    filter_values = (variant_dataset[field] == value).data.compute()
    variant_dataset_filtered = variant_dataset.isel(variants=filter_values)
    return variant_dataset_filtered


def variant_positions(positions):
    return list(positions)


def unique_allele_counts_in_window(gt):
    unique, index, counts = np.unique(gt, axis=1, return_counts=True, return_index=True)
    # Find index with the missing or het
    alleles_with_missing = []
    alleles_with_het = []
    for i in range(len(index)):
        if -1 in (gt[:, index[i]].compute()):
            alleles_with_missing.append(i)
        if True in gt[:, int(index[i])].is_het().compute():
            alleles_with_het.append(i)

    return counts, alleles_with_missing, alleles_with_het


def calculate_stats(variant_dataset, window_length, step):
    pos = variant_dataset["variants"].data

    # Count variants in each window
    n_variants, windows = allel.windowed_count(
        pos, size=window_length, step=step
    )

    # Exclude windows with no variants
    index_with_variants = [i for i, var in enumerate(n_variants) if var != 0]
    window_with_variants = [list(windows[i]) for i in index_with_variants]

    # Find windows with unique variants
    if len(window_with_variants) > 1:
      positions, windows, counts = allel.windowed_statistic(
          pos, pos, statistic=variant_positions, windows=window_with_variants
      )
      unique_var, unique_var_index = np.unique(positions, return_index=True)
      unique_windows = [list(window_with_variants[i]) for i in unique_var_index]
    else:
      unique_windows = window_with_variants
    print('Number of unique windows within region: ', len(unique_windows))
    # Count occurances of each unique allele
    values = allel.GenotypeDaskArray(variant_dataset["call_genotype"].data.astype(int))

    allele_counts, windows, counts = allel.windowed_statistic(
        pos,
        values,
        statistic=unique_allele_counts_in_window,
        windows=unique_windows,
        fill=[0, None, None],
    )
    n_variants, windows = allel.windowed_count(
        pos, windows=unique_windows
    )

    return n_variants, allele_counts, windows

In [15]:
def evaluate_marker_options(
    variant_dataset, chrom, region_df, window_length=200, step=50
):

    # Filter variants to chromosome and set index
    variant_dataset = filter_variants(variant_dataset, "variant_chrom", chrom)
    variant_dataset = variant_dataset.set_index(
        variants="variant_position", samples="sample_id"
    )

    # Find core region boundaries for chromosome
    core_region_df = region_df.loc[
        (region_df.chrom == chrom) & (region_df.name == "Core")
    ]

    biallelic_counts = []
    unique_allele_counts = []
    unique_alleles_with_missing = []
    unique_alleles_with_het = []
    window_start = []
    window_end = []
    variant_counts = []

    # For each region
    for index, row in core_region_df.iterrows():
        print(f"starting sliding window for region: {row.chromStart}-{row.chromEnd}")

        # Restrict variants to region
        variant_dataset_region = variant_dataset.sel(
            variants=slice(row.chromStart, row.chromEnd)
        )

        # STATS
        n_variants, allele_counts, windows = calculate_stats(
            variant_dataset_region, window_length, step
        )

        # Concatenate results
        window_start = window_start + list(windows[:, 0])
        window_end = window_end + list(windows[:, 1])
        variant_counts = variant_counts + list(n_variants)
        unique_allele_counts = unique_allele_counts + list(
            allele_counts[:, 0]
        )
        unique_alleles_with_missing = unique_alleles_with_missing + list(
            allele_counts[:, 1]
        )
        unique_alleles_with_het = unique_alleles_with_het + list(allele_counts[:, 2])

    return (
        variant_counts,
        unique_allele_counts,
        unique_alleles_with_missing,
        unique_alleles_with_het,
        window_start,
        window_end,
    )

### Main execution

This is the main cycle of the notebook. Be aware that the execution across the entire core genome can require significant time (multiple hours). Results are stored per chromosome and then aggregated into a single file, allowing to resume computation from the last completed chromosome, if needed.

In [16]:
results_directory = '../sliding_window_results'

In [17]:
# List of chromosomes to iterate through - the range can be 
chromosomes = [f'PvP01_{chr_num+1:02}_v1' for chr_num in range(14)]

In [ ]:
print('Chromosomes to be scanned: ', chromosomes)

for chrom in chromosomes:
    print('Chromosome:',chrom)
    # Calculate window stats
    (
        variant_counts,
        unique_allele_counts,
        unique_alleles_with_missing,
        unique_alleles_with_het,
        window_start,
        window_end,
    ) = evaluate_marker_options(variant_dataset_filtered, chrom, pv4_regions)

    # Format data
    df = pd.DataFrame(
        data={
            "chrom": chrom,
            "window_start": window_start,
            "window_end": window_end,
            "variant_counts": variant_counts,
            "unique_allele_counts": unique_allele_counts,
            "unique_alleles_with_missing_index": unique_alleles_with_missing,
            "unique_alleles_with_het_index": unique_alleles_with_het,
        }
    )

    # Calculate entropy and heterozygosity
    unique_allele_count = []
    unique_allele_freqs = []
    entropy = []
    het = []
    df_with_stats = df.copy()
    for index, row in df.iterrows():
        gt_counts = row.unique_allele_counts
        n_alleles = len(gt_counts)
        gt_freqs = gt_counts/sum(gt_counts)

        unique_allele_freqs.append(list(gt_freqs))
        unique_allele_count.append(n_alleles)
        entropy.append(-np.sum(gt_freqs * np.log(gt_freqs)))
        het.append(1.0 - np.sum(gt_freqs ** 2))
    df_with_stats["unique_allele_frequencies"] = unique_allele_freqs
    df_with_stats["unique_allele_count"] = unique_allele_count
    df_with_stats["entropy"] = entropy
    df_with_stats["het"] = het

    # Output to csv
    df_with_stats.to_csv(os.path.join(results_directory,f"{chrom}_windowed_stats.csv")

Chromosomes to be scanned:  ['PvP01_01_v1' 'PvP01_02_v1' 'PvP01_03_v1' 'PvP01_04_v1' 'PvP01_05_v1'
 'PvP01_06_v1' 'PvP01_07_v1' 'PvP01_08_v1' 'PvP01_09_v1' 'PvP01_10_v1'
 'PvP01_11_v1' 'PvP01_12_v1' 'PvP01_13_v1' 'PvP01_14_v1']
Chromosome: PvP01_01_v1
starting sliding window for region: 116542-677963
Number of unique windows within region:  401


The final step is to collate all the chromosome-level files into a single file, resulting in an output file that contains each identfied window along with all of its summary statistics called **genome_stats.csv**, which can be found in the **precomputed** directory.

In [61]:
all_files = glob.glob(os.path.join(results_directory , "*_windowed_stats.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0)
    li.append(df)
results_df = pd.concat(li, ignore_index=True)
results_df.to_csv(os.path.join(results_directory , "genome_stats.csv"))